In [ ]:
import kagglehub

import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

import matplotlib.pyplot as plt

import cv2

In [ ]:
# Download latest version
path = kagglehub.dataset_download("ashwingupta3012/human-faces")

print("Path to dataset files:", path)

# 다운로드한 데이터셋 경로 확인
dataset_path = path + '/Humans'  # kagglehub에서 제공한 경로를 사용
print("Dataset files:", os.listdir(dataset_path))

In [ ]:
# Download latest version
path2 = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path2)

# 다운로드한 데이터셋 경로 확인
dataset_path2 = path2 + '/img_align_celeba/img_align_celeba'  # kagglehub에서 제공한 경로를 사용
print("Dataset files:", os.listdir(dataset_path2))

In [ ]:
# 이미지 크기 지정
img_size = 64  # 원하는 크기 (예: 64x64)

def preprocess_images_with_opencv(dataset_path, img_size):
    i = 0

    # 데이터셋 폴더 내 이미지 파일 리스트
    image_files = [f for f in os.listdir(dataset_path) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]
    images = []

    if not image_files:
        print("Warning: No image files found in the dataset path.")
        return np.array([])

    for img_file in image_files:
        i += 1

        if i % 100 == 0:
            print(f"{i}/{str(len(image_files))}")
            if i == 20000:  # 시간상 1000개의 데이터에 대해서만 수행
                #pass
                break

        img_path = os.path.join(dataset_path, img_file)

        #컬러
        #img = cv2.imread(img_path)  # 이미지를 읽기
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환

        #흑백
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 흑백으로 이미지를 읽기

        img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)  # 크기 조정
        img = img.astype(np.float32)  # 데이터 타입 변환
        img = (img - 127.5) / 127.5  # [-1, 1]로 정규화

        #흑백
        img = np.expand_dims(img, axis=-1)  # (height, width) -> (height, width, 1) 형태로 변환
        images.append(img)

    return np.array(images)

In [ ]:
images_cv1 = preprocess_images_with_opencv(dataset_path, img_size)
print("Processed images shape:", images_cv1.shape)

plt.imshow(images_cv1[100], cmap='gray')  # 첫 번째 이미지 출력
plt.axis('off')  # 축을 제거
plt.show()
print(images_cv1.shape)

In [ ]:
images_cv2 = preprocess_images_with_opencv(dataset_path2, img_size)
print("Processed images shape:", images_cv2.shape)

plt.imshow(images_cv2[100], cmap='gray')  # 첫 번째 이미지 출력
plt.axis('off')  # 축을 제거
plt.show()
print(images_cv2.shape)

In [ ]:
images_cv = np.concatenate((images_cv1, images_cv2), axis=0)

In [ ]:
print(images_cv.shape)
plt.imshow(images_cv[25000], cmap='gray')  # 첫 번째 이미지 출력
plt.axis('off')  # 축을 제거
plt.show()

In [ ]:
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        # Dense layer to reshape latent vector into a 4x4x512 tensor
        layers.Dense(4 * 4 * 512, use_bias=False, input_shape=(latent_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((4, 4, 512)),  # Reshaping to 4x4x512

        # First transpose convolution to upsample to 8x8x256
        layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Second transpose convolution to upsample to 16x16x128
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Third transpose convolution to upsample to 32x32x64
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Final transpose convolution to upsample to 64x64x1
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

def build_critic():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same', input_shape=[img_size, img_size, 1]),
        layers.LeakyReLU(),

        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Flatten and output a single value
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(1)
    ])
    return model

In [ ]:
def gradient_penalty(critic, real_images, fake_images):
    batch_size = tf.shape(real_images)[0]
    fake_images = fake_images[:batch_size]

    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
    interpolated = alpha * real_images + (1 - alpha) * fake_images

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        interpolated_output = critic(interpolated)

    gradients = tape.gradient(interpolated_output, [interpolated])[0]
    gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
    penalty = tf.reduce_mean((gradients_norm - 1.0) ** 2)

    return penalty

In [ ]:
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

def critic_loss(real_output, fake_output):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

In [ ]:
@tf.function
def train_step(real_images, generator, critic, gen_optimizer, crit_optimizer, latent_dim, gp_weight=10):
    noise = tf.random.normal([BATCH_SIZE, latent_dim])

    # 판별자(critic) 학습
    with tf.GradientTape() as crit_tape:
        fake_images = generator(noise, training=True)
        real_output = critic(real_images, training=True)
        fake_output = critic(fake_images, training=True)
        print("Real images shape:", real_images.shape)
        print("Fake images shape:", fake_images.shape)
        gp = gradient_penalty(critic, real_images, fake_images)
        crit_loss = critic_loss(real_output, fake_output) + gp_weight * gp

    crit_gradients = crit_tape.gradient(crit_loss, critic.trainable_variables)
    crit_optimizer.apply_gradients(zip(crit_gradients, critic.trainable_variables))

    # 생성자(generator) 학습
    with tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)
        fake_output = critic(fake_images, training=True)
        gen_loss = generator_loss(fake_output)

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))

    return crit_loss, gen_loss

In [ ]:
def train(generator, critic, dataset, epochs, latent_dim):
    gen_optimizer = tf.keras.optimizers.Adam(1e-4)
    crit_optimizer = tf.keras.optimizers.Adam(1e-4)

    for epoch in range(epochs):
        for real_images in dataset:
            crit_loss, gen_loss = train_step(real_images, generator, critic, gen_optimizer, crit_optimizer, latent_dim)
        print(f"Epoch {epoch+1}, Critic Loss: {crit_loss:.4f}, Generator Loss: {gen_loss:.4f}")

In [ ]:
data = images_cv
BATCH_SIZE = 64
BUFFER_SIZE = 60000
dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset

In [ ]:
LATENT_DIM = 100
EPOCHS = 500

generator = build_generator(LATENT_DIM)
critic = build_critic()

train(generator, critic, dataset, EPOCHS, LATENT_DIM)

In [ ]:
def generate_and_save_images(generator, latent_dim, num_examples=16):
    noise = tf.random.normal([num_examples, latent_dim])
    generated_images = generator(noise, training=False)
    generated_images = (generated_images + 1) / 2.0  # [0, 1]로 정규화

    fig, axes = plt.subplots(4, 4, figsize=(6, 6))
    for i, ax in enumerate(axes.flat):
        ax.imshow(generated_images[i, :, :, 0], cmap='gray')
        ax.axis('off')
    plt.show()

generate_and_save_images(generator, LATENT_DIM)